# React Agent 

- React (Reasoning + Action) in the Loop:
    + **Reasoning**: LLM anslysis the user's input, reasons about how to solve the problem, and decides whether to call a tool.
    + **Action**: If needed, the agent invokes one or more tools and receives results (observations).
    + **Observation**: The tool's ouput is fed back to the LLM for further resoning or to geneerate a final response.
    + The loop continues until the LLM determines no further tool calls are needed and provides a final answer
- State Graph:
    + Uses a graph to manage the workflow, consisting of:
        + Nodes: Represent actions (e.g., calling the LLM , executng a tool).
        + Edges: Define the control flow between nodes.
        + State: A object storing the agent's current state, typically a list of messages of additional information like conversation history or intermediate steps.
- Human-in-the-Loop(Optinal):
    - Suport integrating human feedback into the loop to review, approve, or edit eht agent's response before final output
- Memory:
    - By default, the ReAct Agent is stateless (no conversation history). You can add a checkpoint to store conversation history, enabling the agent to respond based on prior context.

In Langgraph:
```
def create_react_agent(
    model: Union[str, LanguageModelLike],
    tools: Union[Sequence[Union[BaseTool, Callable, dict[str, Any]]], ToolNode],
    *,
    prompt: Optional[Prompt] = None,
    response_format: Optional[
        Union[StructuredResponseSchema, tuple[str, StructuredResponseSchema]]
    ] = None,
    pre_model_hook: Optional[RunnableLike] = None,
    post_model_hook: Optional[RunnableLike] = None,
    state_schema: Optional[StateSchemaType] = None,
    config_schema: Optional[Type[Any]] = None,
    checkpointer: Optional[Checkpointer] = None,
    store: Optional[BaseStore] = None,
    interrupt_before: Optional[list[str]] = None,
    interrupt_after: Optional[list[str]] = None,
    debug: bool = False,
    version: Literal["v1", "v2"] = "v2",
    name: Optional[str] = None,
) -> CompiledGraph:
```

Limitation: 
    - If workflow complicate -> mabe need create new custom workflow instead of use reactAgent.
    


In [1]:
import sys
sys.path.append('/home/ai/TanND22/learn_agent/')

In [2]:
import logging
import math
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from src.config.llms import base_model  
from typing_extensions import Annotated, TypedDict
from langgraph.graph.message import add_messages
from langgraph.prebuilt.chat_agent_executor import AgentState
from typing import Sequence
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langgraph.managed import IsLastStep, RemainingSteps
class CustomerState(TypedDict):
    """The state of the agent."""

    messages: Annotated[Sequence[BaseMessage], add_messages]
    expression: str
    is_last_step: IsLastStep
    remaining_steps: RemainingSteps


from loguru import logger

# Prompt template for the math agent
PROMPT_TEMPLATE = """
You are a mathematical assistant capable of solving complex expressions involving logarithms, sine, and cosine functions.
Your task is to break down the input expression into smaller parts, solve each part step-by-step, and provide the final result.
Use the provided tools to perform specific calculations:
- `calculate_log(value, base)` for logarithm calculations (default base is e)
- `calculate_sin(value)` for sine calculations (input in radians)
- `calculate_cos(value)` for cosine calculations (input in radians)
- `calculate_sum(values)` for summing a list of values
-  `calculate_div(a, b) ` for div(a,b)
- `convert_degree_to_radian(values)` for convert of list degree to radians if needed


Input expression: {expression}

Steps to solve:
1. Parse the expression to identify components (e.g., log, sin, cos, sum, convert).
2. Break down the expression into smaller sub-expressions.
3. Use the appropriate tool for each sub-expression.
4. Combine the results to compute the final answer.
5. If the expression is ambiguous or cannot be parsed, return an error message.
6. You can caculator Tan if needed by use (sin/cos)

Provide the final answer in the format:
Final Answer: [result]
"""

# Define mathematical tools
@tool
async def calculate_log(value: float, base: float = math.e) -> float:
    """Calculate the logarithm of a value with a specified base."""
    logger.info(f"[Call Log function] log({value}, {base})")
    return math.log(value, base)

@tool
async def calculate_div(a: float, b: float) -> float:
    """Calculate the div of two values"""
    logger.info(f"[Call Div function]: {a}/{b})")
    return a/b

@tool
async def calculate_sin(value: float) -> float:
    """Calculate the sine of a value (in radians)."""
    logger.info(f"[Call Sin function] sin({value})")
    return math.sin(value)

@tool
async def calculate_cos(value: float) -> float:
    """Calculate the cosine of a value (in radians)."""
    logger.info(f"[Call Cos function] cos({value})")
    return math.cos(value)

@tool
async def calculate_sum(values: list[float]) -> float:
    """Calculate the sum of a list of values."""
    logger.info(f"[Call Sum function] sum({values})")
    return sum(values)

@tool
async def convert_degree_to_radian(values: list[float]) -> float:
    """Convert a list of degree values to radians."""
    logger.info(f"[Call Convert function] convert_degree_to_radian({values})")
    return [math.radians(value) for value in values]

# Apply prompt template
def apply_prompt_template(prompt: str, state: AgentState) -> list:
    """
    Apply template variables to a prompt template and return formatted messages.

    Args:
        prompt: Prompt template to use
        state: Current agent state containing messages

    Returns:
        List of messages with the system prompt as the first message
    """
    try:
        # Extract expression from the last user message
        expression = state.get("expression")
        system_prompt = prompt.format(expression=expression)
        return [{"role": "system", "content": system_prompt}] + state["messages"]
    except Exception as e:
        logger.error(f"Error applying template: {e}")
        return [{"role": "system", "content": f"Error: Invalid expression provided. {e}"}] + state["messages"]

# Create math agent
def create_math_agent():
    """Create a math agent for solving complex mathematical expressions."""
    tools = [calculate_log, calculate_sin, calculate_cos, calculate_sum, convert_degree_to_radian, calculate_div]
    return create_react_agent(
        name="MathAgent",
        model=base_model,
        tools=tools,
        prompt=lambda state: apply_prompt_template(PROMPT_TEMPLATE, state),
        state_schema=CustomerState
    )

In [3]:
math_agent = create_math_agent()
# Initialize state with the expression as a user message
initial_state = {
    "expression": "Log(1+cos(23 degree) + 10 + sin(1 degree))"
}
config = {}  # Add any specific configuration if needed

async for s in math_agent.astream(input=initial_state, config=config, stream_mode="values"):
    # try:
    #     if isinstance(s, dict) and "messages" in s and s["messages"]:
    #         message = s["messages"][-1]
    #         if isinstance(message, dict) and "content" in message:
    #             print(f"Agent output: {message['content']}")
    #         else:
    #             logger.debug(f"Non-message output: {message}")
    #         message.pretty_print()
    #     else:
    #         logger.debug(f"Stream output: {s}")
    # except Exception as e:
    #     logger.error(f"Error processing stream output: {e}")
    #     print(f"Error processing output: {str(e)}")
    try:
        if isinstance(s, dict) and "messages" in s and s["messages"]:
            message = s["messages"][-1]
            if isinstance(message, AIMessage) and message.content:
                print(f"Agent reasoning: {message.content}")
            else:
                logger.debug(f"Other message: {message}")
        else:
            logger.debug(f"Stream output: {s}")
    except Exception as e:
        logger.error(f"Error processing stream output: {e}")
        print(f"Error processing output: {str(e)}")


2025-05-31 10:16:11.757 | DEBUG    | __main__:<module>:30 - Stream output: {'messages': [], 'expression': 'Log(1+cos(23 degree) + 10 + sin(1 degree) + tan(20 degree))'}
2025-05-31 10:16:13 | INFO     | httpx | HTTP Request: POST https://ivysymintegration-openai-eastus-dev.openai.azure.com/openai/deployments/ivysymintegration-gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-05-31 10:16:14.002 | DEBUG    | __main__:<module>:28 - Other message: content='' additional_kwargs={'tool_calls': [{'id': 'call_jYDddENpSyJuvQoca8CScC9J', 'function': {'arguments': '{"values":[23,1,20]}', 'name': 'convert_degree_to_radian'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 472, 'total_tokens': 494, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'mode

Agent reasoning: Final Answer: 2.479816828300002


In [5]:
import math

# Chuyển đổi góc sang radian và tính các giá trị lượng giác
cos_23 = math.cos(math.radians(23))
sin_1 = math.sin(math.radians(1))

# Tính tổng bên trong logarit
total = 1 + cos_23 + 10 + sin_1

# Tính logarit tự nhiên
result = math.log(total)

print(result)

2.4797230095666305
